<a href="https://colab.research.google.com/github/ironcevic/modelling_week11/blob/main/Practical12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Welcome to Google Colab! (yes single l)
Colab is set up in Python.
In Colab, some lines are code and some are comments.
This whole block is a comment because it is encased in quotes.
Comments can also be marked with #.
You can toggle # on and off using "Ctrl + /".
"""

print("Hello, world!") # this is a line of code
# print("This is commented out") # this will not execute as it starts with #

a = 5
# a = "ice cream"
print(a) # is a 5 or ice cream?

"""
You can also run bash commands by pre-appending lines with "!".
Try it below.
There is also a file explorer on the left.
"""

!ls # this is a bash command
!pwd


In [ ]:
"""
Here are some more basic examples.
Try them out by uncommenting them one by one.
"""
import numpy as np # imports the NumPy library

# # Basic printing
# print(1 + 1)           # prints: 2

# # String variables
# my_string = "monster truck"
# print(my_string)       # prints: monster truck

# # 1D arrays
# my_array = np.array([2, 3, 4])
# print(my_array[0])     # prints: 2 (first element)
# print(my_array[-1])    # prints: 4 (last element)

# # 2D arrays
# my_2d_array = np.array([[1, 2, 3], [4, 5, 6]])
# print(my_2d_array[0])        # prints: [1 2 3] (first row)
# print(my_2d_array[0, 1])     # prints: 2 (row 0, column 1)
# print(my_2d_array[:, 0])     # prints: [1 4] (first column)
# print(my_2d_array.T[0])      # prints: [1 4] (first column via transpose)

In [ ]:
"""
Here we will install Quantum Espresso and load the modules we need.
"""

!sudo apt-get update
!sudo apt-get install -y quantum-espresso
!pip install ase
!git clone https://github.com/ironcevic/modelling_week11.git
!mv modelling_week11/* .
!rm -r modelling_week11
from functions import *


In [ ]:
"""
Inspect cumulene.scf.in by double-clicking on it.
How many k-points will this calculation include?

Using the provided code, convert the geometry into .xsf .xyz files and visualise
them on your local computer. This is done by creating a python object, atoms,
and writing it to a file. After executing the code below, download
"unitcell.xsf" and "supercell.xyz" and open them using Vesta and Avogadro,
respectively.
"""

atoms = read('cumulene.scf.in', format='espresso-in') # load geometry
write('unitcell.xsf', atoms) # write the atoms object as an xsf
supercell = atoms.repeat((10, 1, 1)) # create supercell object
write('supercell.xyz', supercell) # write supercell object as xyz



In [ ]:
"""
Run a single-point calculation by executing the code below.
This will take ~1 min 50 s.
Inspect the output file and determine the Fermi energy and the total energy.
"""

!pw.x < cumulene.scf.in > cumulene.scf.out


In [ ]:
"""
Now we shall run a density of states (DOS) post-processing calculation.
Inspect cumulene.dos.in.
How many eV in each direction relative to E_F does it include?
Run the code below. Which files does it produce?
"""

!dos.x < cumulene.dos.in > cumulene.dos.out


In [ ]:
"""
Using the provided code, create the dos object.
How does it compare to the cumulene.dos file?
Plot the density of states using the provided code.
Is cumulenic \textit{trans}-polyacetylene an insulator or a conductor?
"""

dos = np.genfromtxt("cumulene.dos", skip_header=1) # create the dos object

print(dos[0]) # what does this print?
print(dos.T[0]) # how about this?

fermi_level = -2.4177 # find in the scf output
energy_limits = [-8, 3] # where do we plot the energy

# this below prints the DOS
plt.plot(dos[:, 0], dos[:, 1], color = 'k') # plot the dos as a black line
plt.axvline(fermi_level, linestyle='dashed', color = "k") # fermi energy
# the next two lines make a fill with a colour depending on occupancy
plt.fill_between(dos[:, 0], dos[:, 1], where=(dos[:, 0] < fermi_level),
                 facecolor=colours["blue"], alpha=0.5, label='occupied')
plt.fill_between(dos[:, 0], dos[:, 1], where=(dos[:, 0] >= fermi_level),
                 facecolor=colours["orange"], alpha=0.5, label='unoccupied')
plt.xlabel("energy (eV)")
plt.ylabel("density of states")
plt.xlim(energy_limits)
plt.ylim(0, 3)
plt.legend(loc="upper left", frameon=False, bbox_to_anchor=(0.05, 1))
plt.show()

In [ ]:
"""
Let us now plot the band structure. First we run a post-processing calculation.
Run the code below. Which files does it produce?
"""

!bands.x < cumulene.bands.in > cumulene.bands.out


In [ ]:
"""
Let us now plot the band structure.
Create the bands object using the provided code.
Compare it to the cumulene.bands.dat.gnu file.
Plot the band structure using the provided code.
Is cumulenic trans-polyacetylene an insulator or a conductor?
Using the provided code, compute the VB and CB effective masses.
"""

bands = np.genfromtxt("cumulene.bands.dat.gnu") # create the bands object
print(np.shape(bands)) # what is its shape? compare with the .gnu file
bands = np.split(bands, 9) # split it into 9 bands
print(np.shape(bands)) # what is its shape now?
print(bands[0]) # try printing the energies of the first band!

fermi_level = # find in the scf output
energy_limits = [-8, 3] # the energy range to plot

# here below we shall plot the band structure
for band in bands:
    x = band[:, 0]
    y = band[:, 1]
    # can you figure out what the four lines below do?
    if np.all(y < fermi_level):
        plt.plot(x, y, color=colours["blue"])
    else:
        plt.plot(x, y, color=colours["orange"])
plt.ylim(energy_limits)
plt.xlim(0, 0.5)
plt.axhline(y=fermi_level, color='k', linestyle='--')
plt.xlabel(r"$k$-point")
plt.ylabel("energy (eV)")
plt.annotate("occupied", xy=(0.01, fermi_level-0.8), color=colours["blue"])
plt.annotate("unoccupied", xy=(0.01, fermi_level+0.5), color=colours["orange"])
plt.show()

# Determine the effective masses.
mass_vb = effective_mass(bands[4][:, 0], bands[4][:, 1], a=2.46, n_points=5)
mass_cb = effective_mass(bands[5][:, 0], bands[5][:, 1], a=2.46, n_points=5)
print(f"Valence band effective mass is {np.abs(np.round(mass_vb, 4))}.")
print(f"Conduction band effective mass is {np.abs(np.round(mass_cb, 4))}.")


In [ ]:
"""
All these results were obtained for a cumulenic geometry.
Let's now relax the geometry and do it all again.
Inspect polyene.relax.in. Which new keywords does it have?
How many k-points will it include?
Insert a desymmetrised geometry in polyene.relax.in and run the code below.
What do you expect to get?
With a reasonable starting geometry this takes 4-5 min.
"""

!pw.x < polyene.relax.in > polyene.relax.out


In [ ]:
"""
Inspect polyene.relax.out. Paste the optimised geometry into cumulene.scf.in.
Visualise the geometry by adapting the code from the cumulene section.
"""

atoms = read('polyene.scf.in', format='espresso-in') # load geometry
write('unitcell-poly.xsf', atoms) # write the atoms object as an xsf
supercell = atoms.repeat((10, 1, 1)) # create supercell object
write('supercell-poly.xyz', supercell) # write supercell object as xyz


In [ ]:
"""
Perform a single-point calculation at the newly optimised geometry by running
the code below. About 2 min.
"""

!pw.x < polyene.scf.in > polyene.scf.out


In [ ]:
"""
By adapting the input files and the code from above, compute and visualise
the density of states and the band structure. Note: you first need to prepare
the polyene.dos.in and polyene.bands.in files.
Is polyenic trans-polyacetylene a metal or an insulator?
Determine the effective mass of polyenic trans-polyacetylene.
"""

!dos.x < polyene.dos.in > polyene.dos.out
!bands.x < polyene.bands.in > polyene.bands.out

